Libraries and Style Files
-------------------------

In [1]:
#we may need some code in the ../python directory and/or matplotlib styles
import sys
sys.path.append('../python/')

#matplotlib for plotting
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.style.use('../mplstyles/stylelib/standard.mplstyle')

Differential Neutron-Nucleus Cross Sections
===========================================

The aim of this work is to compare the new neutron-electron magnetic process to other neutron-induced processes for rare-event background studies. The neutron-nuclear process is the competing process in the low energy regime. Recoil spectra from this process depend on the differential cross section of the neutron-nuclear scattering. Here we explain our processes for obtaining our estimates for these cross sections. 


ENDF6 Files
-----------

The Evaluated Nuclear Database File (ENDF) has been the format for compiling and transmitting the nuclear data for many years. The [[ENDF/B-VIII.0][ENDF/B-VIII]] (2018) is the most recent evaluation we have copied these files in the directory `xn_data/` which includes official ENDF6 files with the extension `.endf` and files with the extension `.txt` that include the total elastic cross section data.

The differential Center-of-Mass (CoM) cross section data is encoded either in tabulated form (a number in the file for each angular point) or in terms of Legendre coefficients. The cross sections that we have used are in terms of the Legendre coefficients according to the equation below. The function $\sigma(\mu,E)$ is the angular cross section in $\mu \equiv \cos{\theta}$ for each incoming neutron energy $E$. 

\begin{equation}
f(\mu,E) = \frac{2\pi}{\sigma_s(E)} \sigma(\mu,E) = \sum_{l=0}^{N_l} \frac{2l+1}{2}a_l(E)P_l(\mu)
\end{equation}


[ENDF/B-VIII]: https://www.nndc.bnl.gov/endf-b8.0/download.html "ENDF/B-VIII.0 Documentation"

Total Elastic Cross Section Data
--------------------------------

The total cross elastic section as a function of neutron energy is needed to find the differential CoM cross sections, as can be seen by the equation above. The evaluated total cross section is included in the `.endf` files, but in the awkward Reich-Moore parameterization. Instead, we downloaded tabular data for the total cross section from the web interface [[Elastic XN][Elastic XN]]. For example, to get the $^{28}$Si elastic cross section we went to the site and filled out the form as follows, selecting only ENDF VIII.0 data.

![ENDF Online](figures/ENDF-VIII.0-Online.png "ENDF Online Data Plotter")

Then, we retrieved the data and did a "quick plot" which shows the total cross section as below.

![ENDF Online](figures/ENDF-VIII.0-28Si-Elastic.png "28-Si Elastic Cross Section from Online Plotter")

To save the data to 

[Elastic XN]: https://www.nndc.bnl.gov/endf/ "cross sections online"